In [15]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import math

In [2]:
userx_df = pd.read_json("../data/MOOCCubeX/entities/user.json",lines=True)

In [4]:
userx_df.sample(5)

,id,name,gender,school,year_of_birth,course_order,enroll_time
649248,U_11828259,冯健,1.0,内蒙古工业大学,NaN,[697018],[2020-09-17 21:38:42]
2605809,U_29348359,,1.0,,NaN,"[881397, 697748, 948411]","[2020-04-13 20:06:54, 2020-04-13 20:07:12, 202..."
1147715,U_15086627,楚轩大校,2.0,,NaN,[840084],[2019-12-18 08:04:26]
3223906,U_33715166,,2.0,,NaN,[2019328],[2020-09-25 10:10:17]
1220017,U_15345552,Tree,1.0,,NaN,[735155],[2020-01-05 13:59:37]


In [7]:
userx_df = userx_df[userx_df["course_order"].apply(lambda x:len(x) >= 3)]
userx_df["num_course_taken"] = userx_df["course_order"].apply(len)
userx_df.head()

,id,name,gender,school,year_of_birth,course_order,enroll_time,num_course_taken
1,U_24,王帅国,1.0,清华大学,6558.0,"[597214, 605512, 597211, 597314, 597208, 62950...","[2019-05-20 16:06:48, 2019-05-24 19:34:43, 201...",65
3,U_53,于歆杰,1.0,清华大学,1973.0,"[696679, 1704639, 943255, 1729417, 682164, 177...","[2020-03-01 21:24:30, 2020-03-12 16:17:02, 202...",8
4,U_54,马昱春,2.0,清华大学,NaN,"[682442, 682164, 1748240, 1778890, 1829031, 17...","[2019-10-09 02:17:49, 2019-11-08 00:49:03, 202...",9
7,U_69,培源_,1.0,清华大学,NaN,"[375775, 375778, 584329, 584313, 597313, 59722...","[2019-02-26 19:21:24, 2019-02-28 18:12:14, 201...",18
8,U_90,刘俊洋,0.0,qinghua,NaN,"[676664, 707135, 696968, 696994, 1824921]","[2020-04-22 18:23:28, 2020-04-22 21:12:21, 202...",5


In [8]:
userx_mean = userx_df['num_course_taken'].mean()
userx_std = userx_df['num_course_taken'].std()
userx_max = userx_df['num_course_taken'].max()
userx_min = userx_df['num_course_taken'].min()

In [9]:
print("Promedio: " + str(userx_mean))
print("Std: " + str(userx_std))
print("Max: " + str(userx_max))
print("Min: " + str(userx_min))
print("Mediana: " + str(userx_df['num_course_taken'].median()))
print("Moda: " + str(userx_df['num_course_taken'].mode().values[0]))

Promedio: 12.479188054062538
Std: 20.622710510811356
Max: 3715
Min: 3
Mediana: 5.0
Moda: 3


In [13]:
train_test_split = userx_df.drop(columns=["gender","name", "year_of_birth", "school", "enroll_time"])
train_test_split

,id,course_order,num_course_taken
1,U_24,"[597214, 605512, 597211, 597314, 597208, 62950...",65
3,U_53,"[696679, 1704639, 943255, 1729417, 682164, 177...",8
4,U_54,"[682442, 682164, 1748240, 1778890, 1829031, 17...",9
7,U_69,"[375775, 375778, 584329, 584313, 597313, 59722...",18
8,U_90,"[676664, 707135, 696968, 696994, 1824921]",5
...,...,...,...
3330281,U_34712088,"[676664, 1903980, 680936]",3
3330285,U_34712098,"[696946, 677018, 707067, 681005, 947240, 17188...",29
3330288,U_34712103,"[680991, 1916207, 1916220, 735296, 677093, 854...",7
3330289,U_34712108,"[697791, 782490, 799796]",3


In [20]:
train_test_split["num_test"] = train_test_split["num_course_taken"].apply(lambda x: math.ceil(x * 0.2))
train_test_split["test"] = train_test_split.apply(lambda x: x["course_order"][-x["num_test"]:], axis=1)
train_test_split["train"] = train_test_split.apply(lambda x: x["course_order"][:-x["num_test"]], axis=1)
train_test_split

,id,course_order,num_course_taken,num_test,test,train
1,U_24,"[597214, 605512, 597211, 597314, 597208, 62950...",65,13,"[948415, 948410, 1808023, 1906706, 681414, 189...","[597214, 605512, 597211, 597314, 597208, 62950..."
3,U_53,"[696679, 1704639, 943255, 1729417, 682164, 177...",8,2,"[1836502, 1794464]","[696679, 1704639, 943255, 1729417, 682164, 177..."
4,U_54,"[682442, 682164, 1748240, 1778890, 1829031, 17...",9,2,"[1793742, 1906706]","[682442, 682164, 1748240, 1778890, 1829031, 17..."
7,U_69,"[375775, 375778, 584329, 584313, 597313, 59722...",18,4,"[875624, 858543, 813711, 880774]","[375775, 375778, 584329, 584313, 597313, 59722..."
8,U_90,"[676664, 707135, 696968, 696994, 1824921]",5,1,[1824921],"[676664, 707135, 696968, 696994]"
...,...,...,...,...,...,...
3330281,U_34712088,"[676664, 1903980, 680936]",3,1,[680936],"[676664, 1903980]"
3330285,U_34712098,"[696946, 677018, 707067, 681005, 947240, 17188...",29,6,"[674938, 681255, 680870, 697045, 948038, 735324]","[696946, 677018, 707067, 681005, 947240, 17188..."
3330288,U_34712103,"[680991, 1916207, 1916220, 735296, 677093, 854...",7,2,"[854843, 866769]","[680991, 1916207, 1916220, 735296, 677093]"
3330289,U_34712108,"[697791, 782490, 799796]",3,1,[799796],"[697791, 782490]"


In [29]:
train_df = train_test_split[["id", "train"]].explode("train").reset_index(drop=True)
train_df = train_df.rename(columns={"train":"course_id"})
train_df["rating"] = 1
train_df.to_feather("../data/MOOCCubeX/train.feather")
train_df

,id,course_id,rating
0,U_24,597214,1
1,U_24,605512,1
2,U_24,597211,1
3,U_24,597314,1
4,U_24,597208,1
...,...,...,...
6683569,U_34712108,782490,1
6683570,U_34712115,883345,1
6683571,U_34712115,770738,1
6683572,U_34712115,676937,1


In [30]:
test_df = train_test_split[["id", "test"]].explode("test").reset_index(drop=True)
test_df = test_df.rename(columns={"test":"course_id"})
test_df["rating"] = 1
test_df.to_feather("../data/MOOCCubeX/test.feather")
test_df

,id,course_id,rating
0,U_24,948415,1
1,U_24,948410,1
2,U_24,1808023,1
3,U_24,1906706,1
4,U_24,681414,1
...,...,...,...
1983579,U_34712103,854843,1
1983580,U_34712103,866769,1
1983581,U_34712108,799796,1
1983582,U_34712115,1738993,1
